# Fine-tuning HuggingFace Llama with Optical Transformer Transform

This tutorial demonstrates how to:
1. Load a pretrained HuggingFace Llama model
2. Transform it using the optical transformer pass from MASE
3. Run continual fine-tuning on the transformed model

## Overview

The optical transformer transform replaces standard PyTorch modules with their optical equivalents that simulate optical computing behavior. This enables:
- Quantized matrix multiplication that models optical hardware
- Noise-aware training for robust optical neural network deployment

## Requirements

```bash
pip install mase-triton transformers datasets accelerate
```

In [1]:
import torch
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    get_scheduler,
    default_data_collator,
)
from datasets import load_dataset
from torch.utils.data import DataLoader
from itertools import chain
from tqdm.auto import tqdm

from chop.passes.module.transforms.onn.transform import (
    OtLinear,
    OtLlamaAttention,
    OtTransformConfig,
    optical_transformer_module_transform_pass,
)

/home/zz7522/miniconda3/envs/mase/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Load Pretrained HuggingFace Llama Model

We'll use a small Llama model for demonstration. You can replace this with any Llama-based model.

In [2]:
# Model configuration
MODEL_NAME = "AICrossSim/clm-60m"  # Small Llama model for demo
BLOCK_SIZE = 128  # Sequence length (use smaller value for demo)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device: {DEVICE}")

Using device: cuda


In [3]:
# Load model and tokenizer
config = AutoConfig.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    config=config,
    attn_implementation="eager",  # Use eager attention for compatibility
)

print(f"Model type: {type(model).__name__}")
print(f"Number of parameters: {sum(p.numel() for p in model.parameters()):,}")

Model type: LlamaForCausalLM
Number of parameters: 82,101,120


## 2. Configure and Apply Optical Transform

We configure the optical transform with quantization parameters and apply it to the model.

In [4]:
# Create ONN configuration
onn_config = OtTransformConfig.create_default()

# Customize configuration (optional)
onn_config["q_levels"] = 256  # Number of quantization levels
onn_config["q_lut_min"] = 0.020040  # Minimum LUT value
onn_config["q_smooth_factor"] = 0.9  # Statistics smoothing factor
onn_config["q_bypass"] = False  # Set to True to bypass optical quantization

print("ONN Configuration:")
for k, v in onn_config.items():
    print(f"  {k}: {v}")

ONN Configuration:
  q_levels: 256
  q_lut_min: 0.02004
  q_smooth_factor: 0.9
  q_init_seed: 0
  q_bypass: False


In [5]:
# Configure the transform pass
# Use regex patterns to match layer names for transformation
pass_args = {
    "by": "regex_name",
    # Transform all attention layers
    r"model\.layers\.\d+\.self_attn": onn_config,
    # Transform attention projections (Q, K, V, O)
    r"model\.layers\.\d+\.self_attn\.(q|k|v|o)_proj": onn_config,
    # Transform MLP layers
    r"model\.layers\.\d+\.mlp\.(gate|up|down)_proj": onn_config,
}

print("Transform pass arguments configured")

Transform pass arguments configured


In [6]:
# Apply the optical transformer transform
print("Transforming model...")
model = optical_transformer_module_transform_pass(model, pass_args)
print("Model transformed successfully!")

  (q_proj): Linear(in_features=384, out_features=384, bias=False)
  (k_proj): Linear(in_features=384, out_features=128, bias=False)
  (v_proj): Linear(in_features=384, out_features=128, bias=False)
  (o_proj): Linear(in_features=384, out_features=384, bias=False)
) to OtLlamaAttention(
  (q_proj): Linear(in_features=384, out_features=384, bias=False)
  (k_proj): Linear(in_features=384, out_features=128, bias=False)
  (v_proj): Linear(in_features=384, out_features=128, bias=False)
  (o_proj): Linear(in_features=384, out_features=384, bias=False)
)
  (q_proj): Linear(in_features=384, out_features=384, bias=False)
  (k_proj): Linear(in_features=384, out_features=128, bias=False)
  (v_proj): Linear(in_features=384, out_features=128, bias=False)
  (o_proj): Linear(in_features=384, out_features=384, bias=False)
) to OtLlamaAttention(
  (q_proj): Linear(in_features=384, out_features=384, bias=False)
  (k_proj): Linear(in_features=384, out_features=128, bias=False)
  (v_proj): Linear(in_featur

  (q_proj): Linear(in_features=384, out_features=384, bias=False)
  (k_proj): Linear(in_features=384, out_features=128, bias=False)
  (v_proj): Linear(in_features=384, out_features=128, bias=False)
  (o_proj): Linear(in_features=384, out_features=384, bias=False)
) to OtLlamaAttention(
  (q_proj): Linear(in_features=384, out_features=384, bias=False)
  (k_proj): Linear(in_features=384, out_features=128, bias=False)
  (v_proj): Linear(in_features=384, out_features=128, bias=False)
  (o_proj): Linear(in_features=384, out_features=384, bias=False)
)
  (q_proj): Linear(in_features=384, out_features=384, bias=False)
  (k_proj): Linear(in_features=384, out_features=128, bias=False)
  (v_proj): Linear(in_features=384, out_features=128, bias=False)
  (o_proj): Linear(in_features=384, out_features=384, bias=False)
) to OtLlamaAttention(
  (q_proj): Linear(in_features=384, out_features=384, bias=False)
  (k_proj): Linear(in_features=384, out_features=128, bias=False)
  (v_proj): Linear(in_featur

Transforming model...


Model transformed successfully!


In [7]:
# Verify the transformation
def count_transformed_layers(model):
    ot_linear_count = 0
    ot_attn_count = 0
    for name, module in model.named_modules():
        if isinstance(module, OtLinear):
            ot_linear_count += 1
        elif isinstance(module, OtLlamaAttention):
            ot_attn_count += 1
    return ot_linear_count, ot_attn_count

ot_linear, ot_attn = count_transformed_layers(model)
print(f"Transformed layers:")
print(f"  OtLinear: {ot_linear}")
print(f"  OtLlamaAttention: {ot_attn}")

Transformed layers:
  OtLinear: 154
  OtLlamaAttention: 22


## 3. Prepare Dataset for Fine-tuning

We'll use a small subset of a text dataset for demonstration.

In [8]:
# Load a small dataset for demonstration
raw_datasets = load_dataset("wikitext", "wikitext-2-raw-v1")

print(f"Train samples: {len(raw_datasets['train'])}")
print(f"Validation samples: {len(raw_datasets['validation'])}")

Train samples: 36718
Validation samples: 3760


In [9]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],
    desc="Tokenizing",
)

print("Tokenization complete")

Tokenization complete


In [10]:
# Group texts into chunks of block_size
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # Drop the remainder
    total_length = (total_length // BLOCK_SIZE) * BLOCK_SIZE
    # Split into chunks
    result = {
        k: [t[i : i + BLOCK_SIZE] for i in range(0, total_length, BLOCK_SIZE)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    desc=f"Grouping texts in chunks of {BLOCK_SIZE}",
)

print(f"Train chunks: {len(lm_datasets['train'])}")
print(f"Validation chunks: {len(lm_datasets['validation'])}")

Train chunks: 19848
Validation chunks: 2074


In [11]:
# Create DataLoaders
BATCH_SIZE = 4

train_dataloader = DataLoader(
    lm_datasets["train"],
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=BATCH_SIZE,
)

eval_dataloader = DataLoader(
    lm_datasets["validation"],
    collate_fn=default_data_collator,
    batch_size=BATCH_SIZE,
)

print(f"Train batches: {len(train_dataloader)}")
print(f"Eval batches: {len(eval_dataloader)}")

Train batches: 4962
Eval batches: 519


## 4. Setup Training

Configure optimizer, scheduler, and training parameters.

In [12]:
# Training hyperparameters
LEARNING_RATE = 2e-4
WEIGHT_DECAY = 0.01
NUM_EPOCHS = 1  # Use 1 epoch for demo
MAX_TRAIN_STEPS = 100  # Limit steps for demo
WARMUP_STEPS = 10

print("Training configuration:")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Weight decay: {WEIGHT_DECAY}")
print(f"  Max train steps: {MAX_TRAIN_STEPS}")

Training configuration:
  Learning rate: 0.0002
  Weight decay: 0.01
  Max train steps: 100


In [13]:
# Move model to device
model = model.to(DEVICE)

# Set all parameters trainable
for param in model.parameters():
    param.requires_grad = True

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params:,}")

Trainable parameters: 82,101,120


In [14]:
# Setup optimizer with weight decay
no_decay = ["bias", "layer_norm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [
            p for n, p in model.named_parameters()
            if not any(nd in n for nd in no_decay) and p.requires_grad
        ],
        "weight_decay": WEIGHT_DECAY,
    },
    {
        "params": [
            p for n, p in model.named_parameters()
            if any(nd in n for nd in no_decay) and p.requires_grad
        ],
        "weight_decay": 0.0,
    },
]

optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=LEARNING_RATE)
print("Optimizer configured")

Optimizer configured


In [15]:
# Setup learning rate scheduler
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=WARMUP_STEPS,
    num_training_steps=MAX_TRAIN_STEPS,
)

print("LR scheduler configured")

LR scheduler configured


## 5. Training Loop

Run the fine-tuning loop with the transformed optical model.

### Quantization Statistics Warmup

**Important:** The optical transformer layers require calibration of their quantization statistics (min/max values) before they can work correctly. Without this warmup:
- The statistics are initialized to `[inf, -inf]`
- The quantized matmul operations produce NaN values
- Loss and perplexity become NaN

We run a few forward passes in **training mode** to let the layers collect statistics from the data.

In [16]:
# Warmup: Run a few forward passes in training mode to initialize quantization statistics
# This is necessary because the optical transformer layers need to calibrate their
# min/max statistics before they can perform quantized operations correctly.

print("Running warmup to initialize quantization statistics...")
model.train()  # Must be in training mode to update stats
num_warmup_batches = 5

with torch.no_grad():  # No need for gradients during warmup
    for i, batch in enumerate(train_dataloader):
        if i >= num_warmup_batches:
            break
        batch = {k: v.to(DEVICE) for k, v in batch.items()}
        _ = model(**batch)
        print(f"  Warmup batch {i+1}/{num_warmup_batches}")

print("Warmup complete! Quantization statistics initialized.")

Running warmup to initialize quantization statistics...
  Warmup batch 1/5
  Warmup batch 2/5
  Warmup batch 3/5
  Warmup batch 4/5
  Warmup batch 5/5
Warmup complete! Quantization statistics initialized.


In [17]:
import math

def evaluate(model, eval_dataloader, device):
    """Evaluate model and return perplexity."""
    model.eval()
    losses = []
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        losses.append(outputs.loss.item())

    avg_loss = sum(losses) / len(losses)
    perplexity = math.exp(avg_loss)
    return avg_loss, perplexity

In [18]:
# Evaluate before training
print("Evaluating before training...")
eval_loss, eval_ppl = evaluate(model, eval_dataloader, DEVICE)
print(f"Initial - Loss: {eval_loss:.4f}, Perplexity: {eval_ppl:.2f}")

Evaluating before training...
Initial - Loss: 6.1221, Perplexity: 455.84


In [19]:
# Training loop
print("\nStarting training...")
model.train()
completed_steps = 0
train_losses = []

progress_bar = tqdm(range(MAX_TRAIN_STEPS), desc="Training")

for epoch in range(NUM_EPOCHS):
    for step, batch in enumerate(train_dataloader):
        batch = {k: v.to(DEVICE) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss
        train_losses.append(loss.item())

        # Backward pass
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        progress_bar.update(1)
        completed_steps += 1

        # Log progress
        if completed_steps % 20 == 0:
            avg_loss = sum(train_losses[-20:]) / min(20, len(train_losses))
            progress_bar.set_postfix({"loss": f"{avg_loss:.4f}"})

        if completed_steps >= MAX_TRAIN_STEPS:
            break

    if completed_steps >= MAX_TRAIN_STEPS:
        break

print(f"\nTraining completed! Steps: {completed_steps}")


Starting training...


Training: 100%|██████████| 100/100 [00:24<00:00,  4.10it/s, loss=5.2910]


Training completed! Steps: 100


In [20]:
# Evaluate after training
print("\nEvaluating after training...")
eval_loss, eval_ppl = evaluate(model, eval_dataloader, DEVICE)
print(f"Final - Loss: {eval_loss:.4f}, Perplexity: {eval_ppl:.2f}")


Evaluating after training...
Final - Loss: 5.3070, Perplexity: 201.75


## 6. Save the Fine-tuned Model (Optional)

In [21]:
# Uncomment to save the model
# OUTPUT_DIR = "./ot-llama-finetuned"
# model.save_pretrained(OUTPUT_DIR)
# tokenizer.save_pretrained(OUTPUT_DIR)
# print(f"Model saved to {OUTPUT_DIR}")

## Summary

This notebook demonstrated:

1. **Loading a HuggingFace Llama model** using `AutoModelForCausalLM`
2. **Configuring the optical transform** with `OtTransformConfig`
3. **Applying the transform pass** using `optical_transformer_module_transform_pass`
4. **Preparing a dataset** for causal language modeling
5. **Running fine-tuning** with the transformed optical model

### Key Points

- Use `attn_implementation="eager"` when loading the model for compatibility
- The transform pass uses regex patterns to match layer names
- Training mode (`model.train()`) enables statistics updates in optical layers
- The optical quantization adds noise that the model learns to be robust against

### References

- [Optical Transformers Paper](https://arxiv.org/abs/2302.10360)
- MASE ONN Transform: `src/chop/passes/module/transforms/onn/`